## Overview

We have a data set involving 70 binary features (1 means question correct, 0 means question was incorrect) corresponding to the 70 question pretest.

We have access to the scores broken down by chapter for the Objective Assessment (the post test), but we will simply predict the overall score on the assessment (the column 'oa all').

Our goal is to use the pre-test to predict the post test score. This will aid in our ability to identify students who could use our outreach and students who are less likely to need any intervention.

We use 6 different models

1. OLS regression using only overall pretest score (baseline)
2. OLS using all 70 features
3. Elastic Net (regularized linear regression)
4. RandomForest
5. XGBoost

After performing randomized hyperparameter optimization, the Elastic Net and XGBoost models performed with similar mean squared error. 

In [145]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [340]:
col_names = ['pa'+str(i+1) for i in range(70)]+['oa all']+['oa m'+str(i+1) for i in range(7)]
pa_cols = ['pa'+str(i+1) for i in range(70)]

df = pd.read_csv('results-clean.csv',header=None)
df.columns = col_names

df.head(5)

,pa1,pa2,pa3,pa4,pa5,pa6,pa7,pa8,pa9,pa10,...,pa69,pa70,oa all,oa m1,oa m2,oa m3,oa m4,oa m5,oa m6,oa m7
0,1,1,1,1,1,1,1,1,1,1,...,1,1,84,100,100,100,82,82,76,72
1,1,1,1,1,1,1,1,1,1,1,...,1,1,81,86,100,100,82,76,88,59
2,1,1,1,1,1,1,1,1,1,1,...,1,1,84,100,100,100,82,82,76,72
3,1,1,1,1,1,1,1,1,1,1,...,1,1,81,86,100,100,82,76,88,59
4,1,1,1,1,1,1,1,1,1,1,...,1,1,91,100,100,100,100,88,100,72


# model 1: linear model using only overall preassessment score

This is essentially what we were looking at before this project - just using the overall score to predict their score on the assessment.

In [196]:
from sklearn.linear_model import LinearRegression

# sum rows of preassessment columns to get overall preassessment score
X = df[pa_cols].sum(axis=1).values.reshape(-1,1)
y = df['oa all']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=222)

model = LinearRegression()
model.fit(X_train,y_train)

y_pred = model.predict(X_test)

print('train mse:',mean_squared_error(y_train,model.predict(X_train)))
print('test mse:',mean_squared_error(y_test,y_pred))

train mse: 39.805317660468056
test mse: 39.2636433005649


# model 2: linear models using all features

In [217]:
from sklearn.linear_model import LinearRegression

# keep all 70 columns of preassessment
X = df[pa_cols].values
y = df['oa all']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=222)

model = LinearRegression()
model.fit(X_train,y_train)
y_pred = model.predict(X_test)
print('train mse:',mean_squared_error(y_train,model.predict(X_train)))
print('test mse:',mean_squared_error(y_test,y_pred))

train mse: 27.519909738258367
test mse: 36.00626284299438


We see only a minor improvement in test MSE, but significant improvement in train MSE. So model is overfitting. Regularization should help.

# model 3: Elastic Net - linear regression with Lasso/Ridge regularization

In [232]:
from sklearn.linear_model import ElasticNet
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform

X = df[pa_cols].values
y = df['oa all']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=222)

elastic = ElasticNet()
distributions = {'alpha':uniform(loc=0,scale=2),'l1_ratio':uniform(loc=0,scale=1)}

model = RandomizedSearchCV(elastic,distributions,random_state=222,cv=5,n_iter=30)
model.fit(X_train,y_train)
y_pred = model.predict(X_test)

print('train mse:',mean_squared_error(y_train,model.predict(X_train)))
print('test mse:',mean_squared_error(y_test,y_pred))

train mse: 31.265411846870446
test mse: 31.935186988686144


Significant improvement in MSE in both train and test.

In [240]:
model3 = model
model3.best_params_

{'alpha': 0.11089857627317512, 'l1_ratio': 0.14235108803830854}

# model 4: kNN regressor

In [257]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.neighbors import KNeighborsRegressor
from scipy.stats import uniform

X = df[pa_cols].values
y = df['oa all']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=222)

knn = KNeighborsRegressor()

distributions = dict(
    n_neighbors=range(1,30),
    metric = ['minkowski'],
    p = range(2,10)
)

model = RandomizedSearchCV(knn,distributions,n_iter=40,cv=5)

model.fit(X_train, y_train)
    
train_error = mean_squared_error(y_train, model.predict(X_train))
test_error = mean_squared_error(y_test, model.predict(X_test))
    
print('train error: ',train_error)
print('test error:  ',test_error,'\n')

model4 = model

train error:  46.62055335968379
test error:   48.553048916685285 



# model 5 RandomForest

In [274]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor()

distributions = dict(
    n_estimators = range(1,500),
    max_depth = range(1,5),
)

model = RandomizedSearchCV(rf,distributions,cv=8,n_iter=10)
model.fit(X_train, y_train)
    
train_error = mean_squared_error(y_train, model.predict(X_train))
test_error = mean_squared_error(y_test, model.predict(X_test))
    
print('train error: ',train_error)
print('test error:  ',test_error)

model5 = model

train error:  27.17691163964016
test error:   35.88423204115849


Even with cross-validation, we are overfitting. The test MSE is still second best of models so far. Perhaps the regularization parameters of XGBoost can give better results without overfitting.

# model 6: XGBoost - gradient boosted trees

In [335]:
from xgboost import XGBRegressor

xgb = XGBRegressor()

distributions = dict(
    n_estimators=range(2,50),
    max_depth = range(1,10),
    learning_rate=[0.001,0.01,0.1,1],
    gamma=[0.5],
    subsample=[0.8]
)

model = RandomizedSearchCV(xgb,distributions,cv=10,n_iter=30)
model.fit(X_train, y_train)
    
train_error = mean_squared_error(y_train, model.predict(X_train))
test_error = mean_squared_error(y_test, model.predict(X_test))
    
print('train error: ',train_error)
print('test error:  ',test_error)

model.best_params_

train error:  11.722170653652132
test error:   41.09974734546946


{'subsample': 0.8,
 'n_estimators': 42,
 'max_depth': 5,
 'learning_rate': 0.1,
 'gamma': 0.5}

In [320]:
#previous is overfitting, so want to lean higher on regularization
# e.g. higher gamma and lower max depth

from xgboost import XGBRegressor

xgb = XGBRegressor()

distributions = dict(
    n_estimators=range(30,200),
    max_depth = range(1,5),
    learning_rate=[0.001,0.01,0.1,1],
    gamma=[0.01, 0.1,1, 10],
    subsample=[0.7, 0.8, 0.9]
)

model = RandomizedSearchCV(xgb,distributions,cv=10,n_iter=70)
model.fit(X_train, y_train)
    
train_error = mean_squared_error(y_train, model.predict(X_train))
test_error = mean_squared_error(y_test, model.predict(X_test))
    
print('train error: ',train_error)
print('test error:  ',test_error)

model.best_params_

train error:  32.37372610112855
test error:   31.85729034221023


{'subsample': 0.8,
 'n_estimators': 94,
 'max_depth': 1,
 'learning_rate': 0.1,
 'gamma': 1}

In [329]:
# previous model is nearest to best so far
# use normal distributions center around these values to try to see if there is more to gain nearby

from xgboost import XGBRegressor
from scipy.stats import norm

xgb = XGBRegressor()

distributions = dict(
    n_estimators=range(70,130),
    max_depth = range(1,4),
    learning_rate= norm(loc=0.12,scale=0.02),
    gamma=norm(loc=1,scale=0.2),
    subsample=norm(loc=0.75,scale=0.08)
)

model = RandomizedSearchCV(xgb,distributions,cv=10,n_iter=70,random_state=222)
model.fit(X_train, y_train)
    
train_error = mean_squared_error(y_train, model.predict(X_train))
test_error = mean_squared_error(y_test, model.predict(X_test))
    
print('train error: ',train_error)
print('test error:  ',test_error)

model.best_params_

train error:  30.32567446411379
test error:   32.07893175868692


{'gamma': 0.8571050946883807,
 'learning_rate': 0.13094219776517732,
 'max_depth': 1,
 'n_estimators': 98,
 'subsample': 0.6353066011215927}

In [337]:
# splitting the difference between two best previous params

from xgboost import XGBRegressor
from scipy.stats import norm

xgb = XGBRegressor()

distributions = {'subsample': [0.55, 0.60, 0.65,0.70, 0.75, 0.8,],
 'n_estimators': [95,100,105,110,115,120],
 'max_depth': [1,2],
 'learning_rate': [0.1,0.11,0.12,0.13],
 'gamma': [0.85,0.9,0.95,1,1.05,1.1,1.15]
}

model = RandomizedSearchCV(xgb,distributions,cv=10,n_iter=200,random_state=222)
model.fit(X_train, y_train)
    
train_error = mean_squared_error(y_train, model.predict(X_train))
test_error = mean_squared_error(y_test, model.predict(X_test))
    
print('train error: ',train_error)
print('test error:  ',test_error)

model6 = model

model6.best_params_

train error:  31.20197289697522
test error:   32.00793647660632


{'subsample': 0.6,
 'n_estimators': 100,
 'max_depth': 1,
 'learning_rate': 0.1,
 'gamma': 1}